# SQLite Progress Tracker

The purpose of this notebook is to track student progress on their personalized 7 day study plan. It expands on Notebook 03.5 (prototype) by implementing a functional SQLite database that stores tasks,  
tracks completion, and enforces rules around daily progress.

**What this notebook does**
1. Loads or creates the `progress.db` SQLite database.  
2. Defines tables for:
   - **Plans** (plan_id, student_feedback, created_date)  
   - **Tasks** (task_id, plan_id, day_number, description, citation, est_mins, is_completed)  
3. Inserts tasks generated from Notebook 03 into the database.  
4. Provides functions to:
   - Mark tasks as complete.  
   - Query incomplete tasks.  
   - Prevent a student from advancing to the next day until all current day tasks are complete.  
5. Exports reports of progress (completed vs. remaining tasks).  

**Inputs**
- JSON-like study plan output  

**Outputs**
- `progress.db` (SQLite database with plans and tasks).  
- Example queries that show student progress across 7 days.  

03.5 is the bridge notebook where you sketched the outline, and 04 will just introduce the purpose:

1. We now want to store generated plans in a database.

2. The goal is to track tasks day by day and student progress.

3. Tables will likely include plans (overall feedback + plan info) and tasks (individual daily tasks).

4. This allows enforcing rules later like requiring Day 1 to be completed before Day 2 unlocks.

In [1]:
import sqlite3
import json
from datetime import datetime

In [2]:
 
class StudyTracker:
    def __init__(self, db_path="progress.db"):
        self.db_path = db_path
        self._init_db()
    
    def _init_db(self):
        """Create database tables."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''CREATE TABLE IF NOT EXISTS plans (
                plan_id INTEGER PRIMARY KEY,
                student_feedback TEXT,
                created_date TEXT
            )''')
            
            conn.execute('''CREATE TABLE IF NOT EXISTS tasks (
                task_id INTEGER PRIMARY KEY,
                plan_id INTEGER,
                day_number INTEGER,
                description TEXT,
                citation TEXT,
                est_mins INTEGER,
                is_completed BOOLEAN DEFAULT 0,
                FOREIGN KEY (plan_id) REFERENCES plans (plan_id)
            )''')
    
    def load_plan(self, json_data, student_feedback=""):
        """Load study plan from JSON."""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            
            # Create plan
            cursor.execute('INSERT INTO plans (student_feedback, created_date) VALUES (?, ?)',
                         (student_feedback, datetime.now().isoformat()))
            plan_id = cursor.lastrowid
            
            # Add tasks
            for day_data in json_data.get("days", []):
                day_num = day_data["day"]
                for task in day_data["tasks"]:
                    cursor.execute('''INSERT INTO tasks 
                        (plan_id, day_number, description, citation, est_mins) 
                        VALUES (?, ?, ?, ?, ?)''',
                        (plan_id, day_num, task["description"], 
                         task.get("citation", ""), task.get("estimated_minutes", 0)))
            
            return plan_id
    
    def complete_task(self, task_id):
        """Mark task as completed."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('UPDATE tasks SET is_completed = 1 WHERE task_id = ?', (task_id,))
            return conn.total_changes > 0
    
    def get_incomplete_tasks(self, plan_id, day_number=None):
        """Get incomplete tasks."""
        with sqlite3.connect(self.db_path) as conn:
            if day_number:
                cursor = conn.execute('''SELECT task_id, day_number, description, est_mins
                    FROM tasks WHERE plan_id = ? AND day_number = ? AND is_completed = 0''',
                    (plan_id, day_number))
            else:
                cursor = conn.execute('''SELECT task_id, day_number, description, est_mins
                    FROM tasks WHERE plan_id = ? AND is_completed = 0''', (plan_id,))
            
            return [{"task_id": r[0], "day": r[1], "description": r[2], "est_mins": r[3]} 
                    for r in cursor.fetchall()]
    
    def can_advance_to_day(self, plan_id, target_day):
        """Check if student can advance to target day."""
        incomplete = []
        for day in range(1, target_day):
            incomplete.extend(self.get_incomplete_tasks(plan_id, day))
        return len(incomplete) == 0, incomplete
    
    def get_progress_report(self, plan_id):
        """Generate progress report."""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute('''SELECT 
                COUNT(*) as total,
                SUM(is_completed) as completed,
                SUM(est_mins) as total_mins,
                SUM(CASE WHEN is_completed THEN est_mins ELSE 0 END) as completed_mins
                FROM tasks WHERE plan_id = ?''', (plan_id,))
            
            stats = cursor.fetchone()
            total, completed = stats[0], stats[1] or 0
            
            return {
                "total_tasks": total,
                "completed_tasks": completed,
                "progress_percent": round((completed/total*100) if total > 0 else 0, 1),
                "total_minutes": stats[2] or 0,
                "completed_minutes": stats[3] or 0
            }

# Demo usage
def demo():
    # Sample study plan
    plan = {
        "days": [
            {"day": 1, "tasks": [
                {"description": "Learn Python basics", "estimated_minutes": 45},
                {"description": "Practice variables", "estimated_minutes": 30}
            ]},
            {"day": 2, "tasks": [
                {"description": "Study loops", "estimated_minutes": 40}
            ]}
        ]
    }
    
    # Setup
    tracker = StudyTracker()
    plan_id = tracker.load_plan(plan, "Want to learn Python")
    
    print("📚 Study Tracker Demo")
    print(f"Created plan ID: {plan_id}")
    
    # Show Day 1 tasks
    day1_tasks = tracker.get_incomplete_tasks(plan_id, 1)
    print(f"\nDay 1 tasks: {len(day1_tasks)}")
    for task in day1_tasks:
        print(f"  • {task['description']}")
    
    # Complete first task
    tracker.complete_task(day1_tasks[0]['task_id'])
    print(f"✅ Completed: {day1_tasks[0]['description']}")
    
    # Check if can advance
    can_advance, blocking = tracker.can_advance_to_day(plan_id, 2)
    print(f"\nCan advance to Day 2: {can_advance}")
    if blocking:
        print(f"Blocked by {len(blocking)} incomplete tasks")
    
    # Show progress
    report = tracker.get_progress_report(plan_id)
    print(f"\nProgress: {report['completed_tasks']}/{report['total_tasks']} ({report['progress_percent']}%)")

if __name__ == "__main__":
    demo()

📚 Study Tracker Demo
Created plan ID: 1

Day 1 tasks: 2
  • Learn Python basics
  • Practice variables
✅ Completed: Learn Python basics

Can advance to Day 2: False
Blocked by 1 incomplete tasks

Progress: 1/3 (33.3%)
